In [ ]:
import torch
torch.set_grad_enabled(False)

from data import load_data
from experiments import run_debate
from model_wrappers import Llama2Wrapper, Llama3Wrapper

%load_ext autoreload
%autoreload 2

In [ ]:
train_data, test_data = load_data()

In [ ]:
from transformers import BitsAndBytesConfig
# quant_config  = HqqConfig(nbits=8, group_size=64, quant_zero=False, quant_scale=False, axis=0) #axis=0 is used by default
quant_config = BitsAndBytesConfig(load_in_8bit=True)
debater_one = Llama2Wrapper.from_hf("llama2_7b", "meta-llama/Llama-2-7b-chat-hf", 
                attn_implementation='flash_attention_2', 
                quantization_config=quant_config)

In [ ]:
debater_two = judge = Llama3Wrapper.from_hf("llama3_8b", "meta-llama/Meta-Llama-3-8B-Instruct",
            attn_implementation='flash_attention_2', 
            quantization_config=quant_config)

In [ ]:
toks = debater_one.model.tokenizer('Hello', return_tensors='pt')['input_ids'].cuda()
logits = debater_one.model.model.generate(toks, max_new_tokens=1, return_dict_in_generate=True, output_scores=True,
                                          temperature=1.0)['scores'][0][0]

In [ ]:
logits.softmax(dim=0)[2850]

In [ ]:
run_debate(debater_one, debater_two, judge, train_data[:5], "results.json")